# **Practical 2**

In this practical, we will examine classical conditioning and use exploration/exploitation to solve a maze.



# **Conditioning vs Supervised and Online Learning**

We start by looking at the following setting. Suppose there is a dog who is in a room with two lights (one red and one green) which will be on or off depending on whether there is food available on a plate. 

We start by generating the data for training and testing. Note this is effectively an XOr gate, which we use below to obtain the targets

In [13]:

# Import libraries and namespaces
import time
from random import seed
from random import randint
import numpy as np

x_train = np.zeros( (100, 2) )
y_train = np.zeros( 100 )
for i in range(100):
    # seed random number generator with the system clock
    seed(time.clock())
        
    # generate random integers between zero and two
    x_train[i,0] = randint(0,1)
    x_train[i,1] = randint(0,1)
    y_train[i] = np.logical_xor(x_train[i,0],x_train[i,1])
    
x_test = [[1,1],[1,0],[0,1],[0,0]]
y_test = [0,1,1,0]

# Converting to float 32bit
x_train = np.array(x_train).astype(np.float32)
x_test  = np.array(x_test).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
y_test  = np.array(y_test).astype(np.float32)

# Print data split for validation
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(100, 2) (100,)
(4, 2) (4,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  if sys.path[0] == '':


We now try the online learning approach with an observation window, i.e. memory. In this case, the network will train with only a few samples at a time, drawing a new observation at every training step

In [14]:
# Importing libraries and namespaces
import scipy as sci
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Setup the variables
y=np.array([0]).astype(np.float32)
x=np.reshape(np.array([0,0]),(1,2)).astype(np.float32)

# Define a shift for the online learning window
def shift(A, N):
    B = np.empty_like(A)
    if N >= 0:
        B[:N] = np.nan
        B[N:] = A[:-N]
    else:
        B[N:] = np.nan
        B[:N] = A[-N:]
    return B

# Set a learning window size. This is the memory of the system
window = 10;

# Model initialization
Model = MLPClassifier(hidden_layer_sizes=(3,3), max_iter=1, alpha=0.01, #try change hidden layer
                     solver='sgd', verbose=1,  random_state=100) 

# Train our model
cl = np.array(np.unique([0,1]))
y = y_train[range(window)]
x = x_train[range(window),:]

for epochs in range(10000):
    # Sample randomly
    i = randint(0,99)
    # Train the model
    Model.partial_fit(x, y, cl)
    # Do the shift and acquire the next instance
    y = shift(y,1)
    x = shift(x,1)
    y[0] = y_train[i]
    x[0,:] = x_train[i,:]
       

#Show the accuracy
y_pred = Model.predict(x_test)
print('accuracy is ',accuracy_score(y_pred,y_test)) # Print accuracy score

Streaming output truncated to the last 5000 lines.
Iteration 7479, loss = 0.05673298
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 7480, loss = 0.05162775
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 7481, loss = 0.05677899
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 7482, loss = 0.05560960
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 7483, loss = 0.06067972
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 7484, loss = 0.06462467
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 7485, loss = 0.06856963
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 7486, loss = 0.06855647
Training loss did not improve more than tol=0.0

We compare our result with that obtained using supervised learning. In the code below, we have used a very similar setting, with the same data and classifier, but with a batch rather than online sampling strategy. 

In [15]:
# Importing libraries and namespaces
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Model initialization
Model = MLPClassifier(hidden_layer_sizes=(4,4), max_iter=10000, alpha=0.01,
                     solver='sgd', verbose=1,  random_state=100) 

# Train our model
h=Model.fit(x_train,y_train)

# Use our model to predict
y_pred=Model.predict(x_test)

# Scikit for machine learning reporting

print('Classification report')
print(classification_report(y_test,y_pred)) # Print summary report
print('accuracy is ',accuracy_score(y_pred,y_test)) # Print accuracy score


Iteration 1, loss = 0.81434741
Iteration 2, loss = 0.81415213
Iteration 3, loss = 0.81387369
Iteration 4, loss = 0.81352054
Iteration 5, loss = 0.81310027
Iteration 6, loss = 0.81261974
Iteration 7, loss = 0.81208513
Iteration 8, loss = 0.81150204
Iteration 9, loss = 0.81087657
Iteration 10, loss = 0.81021811
Iteration 11, loss = 0.80952390
Iteration 12, loss = 0.80879771
Iteration 13, loss = 0.80804291
Iteration 14, loss = 0.80726255
Iteration 15, loss = 0.80646216
Iteration 16, loss = 0.80564162
Iteration 17, loss = 0.80480282
Iteration 18, loss = 0.80395033
Iteration 19, loss = 0.80308198
Iteration 20, loss = 0.80220314
Iteration 21, loss = 0.80131185
Iteration 22, loss = 0.80041205
Iteration 23, loss = 0.79950256
Iteration 24, loss = 0.79858746
Iteration 25, loss = 0.79766351
Iteration 26, loss = 0.79673630
Iteration 27, loss = 0.79580162
Iteration 28, loss = 0.79486485
Iteration 29, loss = 0.79392246
Iteration 30, loss = 0.79297816
Iteration 31, loss = 0.79203049
Iteration 32, los

# **Action Selection**

We now turn our attention to action selection. To this end, we use a maze as the environment under consideration and proceed to install the required system dependencies


In [16]:
# install required system dependencies
!apt-get install -y xvfb x11-utils  
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyOpenGL==3.1.* \
            PyOpenGL-accelerate==3.1.* \
            gym[box2d]==0.17.* 
!pip install pyglet
!pip install ffmpeg
! pip install pyvirtualdisplay
!pip install Image
!pip install gym-maze-trustycoder83

Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


If the directory vid exists and has videos left over from previous tries, its better to clean it up before continuing.

In [17]:
!mkdir ./vid
!rm ./vid/*.*

mkdir: cannot create directory ‘./vid’: File exists


We now proceed to initialise the monitor wrapper for Gym so we can visualise the maze and the agent on a video

In [18]:
import sys
# import pygame
import numpy as np
# import math
# import base64
# import io
# import IPython
import gym
import gym_maze

# from gym.wrappers import Monitor
# from IPython import display
from pyvirtualdisplay import Display
from gym.wrappers.monitoring import video_recorder

d = Display()
d.start()

# Recording filename
video_name = "./vid/Practical_2.mp4"

# Setup the environment for the maze
env = gym.make("maze-sample-10x10-v0")

# Setup the video
vid = None
vid = video_recorder.VideoRecorder(env,video_name)

# env = gym.wrappers.Monitor(env,'./vid',force=True)
current_state = env.reset()



We now proceed to perform a simple Q-tracking algorithm. The method here is quite sub-optimal since it employs a random choice to pick between exploration and exploitation. It does illustrate, however, how Q-value tracking can be done using the maze.

In [19]:

discount_rate = 0.99


In [20]:
states_dic = {} #dictionary to keep the states/coordinates of the Q table
count = 0
for i in range(10):
    for j in range(10):
        states_dic[i, j] = count
        count+=1
        
n_actions = env.action_space.n

# Initialize the Q-table to 0
Q_table = np.zeros((len(states_dic),n_actions))


# Number of episode we will run
n_episodes = 10

# Maximum of iteration per episode
max_iter_episode = 500

# Initialize the exploration probability to 1
exploration_proba = 0.5

#Exploartion decreasing decay for exponential decreasing
exploration_decreasing_decay = 0.01

# Minimum of exploration prob
max_exploration_proba = 1
min_exploration_proba = 0.01

# Learning rate
lr = 0.1

rewards_per_episode = list()


# Iterate over episodes
for e in range(n_episodes):
    
    # We are not done yet
    done = False
    
    # Sum the rewards that the agent gets from the environment
    total_episode_reward = 0

    for i in range(max_iter_episode): 
        env.unwrapped.render()
        vid.capture_frame()
        current_coordinate_x = int(current_state[0])
        current_coordinate_y = int(current_state[1])
        current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]
        exploration_rate_threshold=np.random.uniform(0,1)
        if  exploration_rate_threshold> exploration_proba:
          action = int(np.argmax(Q_table[current_Q_table_coordinates]))
        else:
          action = env.action_space.sample()

        next_state, reward, done, _ = env.step(action)

        next_coordinate_x = int(next_state[0]) #get coordinates to be used in dictionary
        next_coordinate_y = int(next_state[1]) #get coordinates to be used in dictionary


        # Update our Q-table using the Q-learning iteration
        next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]
        Q_table[current_Q_table_coordinates, action] = (1-lr) *Q_table[current_Q_table_coordinates, action] +lr*(reward + max(Q_table[next_Q_table_coordinates,:]))
    
        total_episode_reward = total_episode_reward + reward
        # If the episode is finished, we leave the for loop
        if done:
            break
        current_state = next_state

    #Show the total episode reward        
    print("Total episode reward:", total_episode_reward)
    
    #Reset enviroment for next episode
    current_state = env.reset()
    #exploration rate decay
    exploration_proba = min_exploration_proba + (max_exploration_proba - min_exploration_proba) * np.exp(-exploration_decreasing_decay*e)
    rewards_per_episode.append(total_episode_reward)

    # Save video episode and close
print("Video successfuly saved.")
vid.close()
vid.enabled = False

Total episode reward: -0.5000000000000003
Total episode reward: -0.5000000000000003
Total episode reward: -0.5000000000000003
Total episode reward: -0.5000000000000003
Total episode reward: -0.5000000000000003
Total episode reward: 0.5539999999999996
Total episode reward: 0.6469999999999998
Total episode reward: -0.5000000000000003
Total episode reward: 0.5189999999999997
Total episode reward: -0.5000000000000003
Video successfuly saved.


In [22]:
print("\n\n********Q-table********\n")
print(Q_table)



********Q-table********

[[-5.28647512e-03 -5.07648209e-03 -5.04384678e-03 -4.95547329e-03]
 [-5.17351804e-03 -4.61454261e-03 -4.91443168e-03 -5.00710988e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e-04 -1.00000000e-04  0.00000000e+00]
 [-1.00000000e-04 -2.01000000e-04 -1.00000000e-04 -2.09000000e-04]
 [-2.09000000e-04 -4.77490000e-04 -4.30410000e-04 -1.90000000e-04]
 [-4.30410000e-04 -7.63629241e-04 -4.83680000e-04 -6.43306900e-04]
 [-4.95770000e-04 -6.65095100e-04 -4.76390000e-04 -7.50300000e-04]
 [-6.54734100e-04 -6.67559000e-04 -4.00000000e-04 -7.04303142e-04]
 [-4.92210039e-03 -4.93498270e-03 -4.94503953e-03 -4.98700429e-03]
 [-4.69935422e-03 -4.87843221e-03 -5.05904812e-03 -4.58024212e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  

We can now play the video using the following code

In [21]:
import base64
import io
from IPython import display

video_name = "./vid/Practical_2.mp4"

video = io.open(video_name, 'r+b').read()
encoded = base64.b64encode(video)

display.display(display.HTML(data="""
  <video alt="test" controls>
  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
  </video>
  """.format(encoded.decode('ascii'))))